### Importing libraries and dataframes

In [2]:
import json
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import random
import time
import sys

In [3]:
playlist_df = pd.read_csv("../Datasets/playlist_df.csv")
features_scaled = pd.read_csv("../Datasets/features_scales_df.csv")
top_100_songs = pd.read_csv("../Datasets/top_hot_100.csv")
features_playlist = pd.read_csv("../Datasets/features_playlist.csv")

### Authentication

In [4]:
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(
    client_id='84728f97b7bd4d948b3f9d1c3105f9f0',
    client_secret='a9506255e5e3422fa20da88022c48c30'))

### Scaling and clustering the songs

In [5]:
scaler = StandardScaler()
features_playlist = playlist_df.iloc[:,4:17]
scaler.fit(features_playlist)
features_scaled = scaler.transform(features_playlist)
features_scaled_df = pd.DataFrame(features_scaled)

In [6]:
kmeans = KMeans()
kmeans.fit(features_scaled)
clusters = kmeans.predict(features_scaled)
playlist_df["cluster"]=clusters

In [7]:
def extract_uri(query):
    results=spotify.search(q=query, limit=10)
    uri=results["tracks"]["items"][0]["uri"]
    song_audio_features = spotify.audio_features(uri)
    return song_audio_features


In [11]:
top_100_songs["name"] = top_100_songs["name"].apply(lambda x: x.lower())
top_100_songs["artist"] = top_100_songs["artist"].apply(lambda x: x.lower())

for song in top_100_songs[["name","artist"]].iterrows():
    user_input = input("Write down a song to get a recommendation: ")
    if user_input.lower() in list(top_100_songs["name"]):
        print("This song is HOT! Wait to get another hot song recommended...")
        random_song = random.choice(top_100_songs["name"]).title()
        song_artist = top_100_songs.loc[top_100_songs["name"] == random.choice(top_100_songs["name"]), "artist"].values[0].title()
        time.sleep(3)
        print("You may also like the song: ", "\033[1m", random_song, "\033[0;0m", "from the artist", "\033[1m", song_artist, "\033[0;0m")
        print("Thanks for using the music recommender.")
        break
    else:
        print("This song is not hot, but wait to get a recommendation based on the audio features of the song...")
        uri_user_input = extract_uri(user_input)
        df_user_input = pd.DataFrame(uri_user_input)
        df_user_input.drop(df_user_input.columns[[11,12,13,14,15]], axis = 1, inplace = True)
        scaler.fit(df_user_input)
        song_scaled = scaler.transform(df_user_input)
        song_cluster = kmeans.predict(song_scaled)
        input_cluster = song_cluster[0]
        match = playlist_df.loc[playlist_df["cluster"] == input_cluster]
        title = match.sample()["names"].values[0]
        playlist_df['artists_clean']=playlist_df['artists'].astype(str)
        artist = playlist_df[playlist_df["names"]==title]["artists_clean"].values[0]
        song_uri = playlist_df[playlist_df["names"]==title]["uri"].values[0]
        song_url = song_uri.strip("spotify/.").replace(":", "/")
        time.sleep(1)

        print("We recommend you the song", "\033[1m", title.title(), "\033[0;0m", "from the artist", "\033[1m", artist.replace(",", " &").replace("[","").replace("]","").replace("'",""), "\033[0;0m")
        print("You can listen to the song here:", "http://open.spotify.com" + song_url)
        time.sleep(2)
        print("Thanks for using the music recommender.")
        break

Write down a song to get a recommendation: mood
This song is HOT! Wait to get another hot song recommended...
You may also like the song:   Holy  from the artist  Cardi B Featuring Megan Thee Stallion 
Thanks for using the music recommender.
